# Случайные леса

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с бустингом, предоставляемым библиотекой `CatBoost`.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека.

In [1]:
from math import log2, sqrt

import inspect
import random
from collections import Counter
from dataclasses import dataclass
from itertools import product
from typing import Callable, List, Tuple, Union

import numpy as np
import numpy.typing as npt
import pandas
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [2]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)


# Этой функцией будут помечены все места, которые необходимо дозаполнить
# Это могут быть как целые функции, так и отдельные части внутри них
# Всегда можно воспользоваться интроспекцией и найти места использования этой функции :)
def todo():
    stack = inspect.stack()
    caller_frame = stack[1]
    function_name = caller_frame.function
    line_number = caller_frame.lineno
    raise NotImplementedError(f"TODO at {function_name}, line {line_number}")


SEED = 0xC0FFEE
set_seed(SEED)

In [3]:
def mode(data):
    counts = Counter(data)
    return counts.most_common(n=1)[0][0]

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bootstrap. Элементы, которые не вошли в новую обучающую выборку, образуют **out-of-bag** выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bootstrap. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [4]:
# Для начала реализуем сами критерии

def count_probs(x: npt.ArrayLike) -> list[float]:
    classes_count = Counter(x)
    return [countt / len(x) for countt in classes_count.values()]


def gini(x: npt.ArrayLike) -> float:
    """
    Calculate the Gini impurity of a list or array of class labels.

    Args:
        x (ArrayLike): Array-like object containing class labels.

    Returns:
        float: Gini impurity value.
    """
    return sum([prob * (1 - prob) for prob in count_probs(x)])


def entropy(x: npt.ArrayLike) -> float:
    """
    Calculate the entropy of a list or array of class labels.

    Args:
        x (ArrayLike): Array-like object containing class labels.

    Returns:
        float: Entropy value.
    """
    return sum([prob * log2(prob) for prob in count_probs(x)])


def gain(left_y: npt.ArrayLike, right_y: npt.ArrayLike, criterion: Callable[[npt.ArrayLike], float]) -> float:
    """
    Calculate the information gain of a split using a specified criterion.

    Args:
        left_y (ArrayLike): Class labels for the left split.
        right_y (ArrayLike): Class labels for the right split.
        criterion (Callable): Function to calculate impurity (e.g., gini or entropy).

    Returns:
        float: Information gain from the split.
    """
    criterion_before_split = criterion(np.concatenate((left_y, right_y)))
    criterion_left = criterion(left_y)
    criterion_right = criterion(right_y)
    len_whole = len(left_y) + len(right_y)
    return criterion_before_split - (len(left_y) / len_whole) * criterion_left - (len(right_y) / len_whole) * criterion_right

In [5]:
@dataclass
class DecisionTreeLeaf:
    classes: np.ndarray

    def __post_init__(self):
        self.max_class = mode(self.classes)


@dataclass
class DecisionTreeInternalNode:
    split_dim: int
    left: Union["DecisionTreeInternalNode", DecisionTreeLeaf]
    right: Union["DecisionTreeInternalNode", DecisionTreeLeaf]


DecisionTreeNode = Union[DecisionTreeInternalNode, DecisionTreeLeaf]

In [6]:
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        boot_X, boot_y, oob_X, oob_y = self._generate_bootstrap_and_oob(X, y)
        self._boot_X = boot_X
        self._boot_y = boot_y
        self._out_of_bag_X = oob_X
        self._out_of_bag_y = oob_y
        self._criterion = gini if criterion == "gini" else entropy
        self._max_depth = max_depth
        self._min_samples_leaf = min_samples_leaf
        self._max_features = max_features if max_features != "auto" else int(sqrt(X.shape[1]))
        self._root = self._build_node(self._boot_X, self._boot_y, 0)

    @property
    def out_of_bag(self) -> Tuple[np.ndarray, np.ndarray]:
        return self._out_of_bag_X, self._out_of_bag_y

    def _generate_bootstrap_and_oob(self, X, y) -> tuple:
        bootstrap_indices = np.random.choice(X.shape[0], size=X.shape[0], replace=True)
        bootstrap_X = X[bootstrap_indices]
        bootstrap_y = y[bootstrap_indices]
        oob_indices = [i for i in range(X.shape[0]) if i not in np.unique(bootstrap_indices)]
        oob_X = X[oob_indices]
        oob_y = y[oob_indices]
        return bootstrap_X, bootstrap_y, oob_X, oob_y

    def _build_node(self, points: np.ndarray, classes: np.ndarray, depth: int) -> DecisionTreeNode:
        if (self._max_depth is not None and depth == self._max_depth) or classes.shape[0] < (
                2 * self._min_samples_leaf) or len(np.unique(classes)) == 1:
            return DecisionTreeLeaf(classes)
        max_gain = 0
        best_split_axis = best_left_ind = best_right_ind = None
        for feature in np.random.choice(points.shape[1], size=self._max_features):
            curr_left_ind = points[:, feature] == 0
            curr_right_ind = ~curr_left_ind
            if sum(curr_left_ind) < self._min_samples_leaf or sum(curr_right_ind) < self._min_samples_leaf:
                continue
            curr_gain = gain(classes[curr_left_ind], classes[curr_right_ind], self._criterion)
            if curr_gain > max_gain:
                max_gain = curr_gain
                best_split_axis = feature
                best_left_ind = curr_left_ind
                best_right_ind = curr_right_ind
        if best_split_axis is None:
            return DecisionTreeLeaf(classes)
        left_node = self._build_node(points[best_left_ind], classes[best_left_ind], depth + 1)
        right_node = self._build_node(points[best_right_ind], classes[best_right_ind], depth + 1)
        return DecisionTreeInternalNode(best_split_axis, left_node, right_node)

    def _predict(self, point: np.ndarray, node: DecisionTreeNode) -> int:
        if isinstance(node, DecisionTreeLeaf):
            return node.max_class

        if point[node.split_dim] == 0:
            return self._predict(point, node.left)
        return self._predict(point, node.right)

    def predict(self, points: np.ndarray) -> np.ndarray:
        return np.array([self._predict(point, self._root) for point in points])

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [7]:
class RandomForestClassifier:

    _n_features: int = None

    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self._criterion = criterion
        self._max_depth = max_depth
        self._min_samples_leaf = min_samples_leaf
        self._max_features = max_features
        self._n_estimators = n_estimators
        self._estimators = []
        self._n_features = None

    @property
    def estimators(self) -> List[DecisionTree]:
        return self._estimators

    @property
    def n_features(self) -> int:
        if self._n_features is None:
            raise RuntimeError("Fit random forest before accessing to number of features properties")
        return self._n_features

    def fit(self, X, y):
        self._n_features = X.shape[1]
        self._estimators = []
        for _ in range(self._n_estimators):
            decision_tree = DecisionTree(
                X, y,
                criterion=self._criterion,
                max_depth=self._max_depth,
                min_samples_leaf=self._min_samples_leaf,
                max_features=self._max_features
            )
            self._estimators.append(decision_tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self._estimators])
        return np.array([mode(predictions[:, i]) for i in range(X.shape[0])])

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [8]:
def accuracy_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = y_true.reshape(-1)
    y_pred = y_pred.reshape(-1)
    return np.mean(y_true == y_pred)


def feature_importance(rfc):
    importances = np.zeros(rfc.n_features)
    
    for tree in rfc.estimators:
        oob_X, oob_y = tree.out_of_bag
        if len(oob_X) == 0:
            continue
            
        y_pred = tree.predict(oob_X)
        original_accuracy = accuracy_score(oob_y, y_pred)
        
        for feature in range(rfc.n_features):
            shuffled_X = oob_X.copy()
            np.random.shuffle(shuffled_X[:, feature])
            
            y_pred_shuffled = tree.predict(shuffled_X)
            shuffled_accuracy = accuracy_score(oob_y, y_pred_shuffled)
            
            importances[feature] += (shuffled_accuracy - original_accuracy)
    importances /= len(rfc.estimators)
    return importances


def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    indices = np.argsort(importance)[::-1][:k]
    return np.array(names)[indices]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [9]:
def synthetic_dataset(size):
    X = [
        (np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, i % 6 == 0, i % 3 == 2, np.random.randint(0, 2))
        for i in range(size)
    ]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)


X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [-1.70452947e-03  2.44826556e-04 -1.54450440e-01 -1.43690023e-01
 -2.94225041e-01 -1.47239289e-03]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [10]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]

    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [11]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [12]:
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i + 1) + ".", name)

Accuracy: 0.6923076923076923
Most important features:
1. vkgames
2. ftp_memes
3. gm.power
4. vkucnie_recepti
5. lhack
6. fucking_humor
7. molodost_bz
8. combovine
9. be.beauty
10. kinohd
11. team
12. theastro
13. morgenshtern666
14. komment.broo
15. authors
16. be.women
17. evil_incorparate
18. modnailru
19. lomai_loogiky
20. pn6


#### Пол

In [13]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i + 1) + ".", name)

Accuracy: 0.8436317780580076
Most important features:
1. otbrosy_pokoleniya
2. leprazo
3. molodost_bz
4. dfilm
5. bog_memes
6. lixie
7. authors
8. vkgames
9. face
10. i.kino
11. v5inf
12. exclusive_muzic
13. tnt
14. ne.poverish
15. best_video_hd
16. rem_shkola
17. sci
18. ohhluul
19. bratishkinoff
20. club128730587


### CatBoost
В качестве аьтернативы попробуем CatBoost.

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [14]:
X, y = synthetic_dataset(1000)

cb_model = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='MultiClass', verbose=False)
cb_model.fit(X, y)
y_pred = cb_model.predict(X)

print("Accuracy:", accuracy_score(y_pred, y))
print("Importance:", cb_model.feature_importances_)

Accuracy: 1.0
Importance: [3.95725570e-03 1.36386250e-03 2.78467422e+01 2.78657214e+01
 4.42806940e+01 1.52125956e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [15]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(
    X_train, y_age_train, y_sex_train, train_size=0.8
)

In [16]:
max_depth = range(1, 10, 3)
min_samples_leaf = range(1, 10, 3)
learning_rate = np.linspace(0.001, 1.0, 5)


def get_best_params(y_train, y_eval):
    best_score, best_params = None, None
    for lr, md, msl in tqdm(list(product(learning_rate, max_depth, min_samples_leaf))):
        model_cat = CatBoostClassifier(
            iterations=100,
            depth=md,
            learning_rate=lr,
            min_data_in_leaf=msl,
            loss_function="MultiClass",
            verbose=False,
            random_seed=SEED
        )
        
        model_cat.fit(X_train, y_train)
        y_pred = model_cat.predict(X_eval)
        curr_score = accuracy_score(y_eval, y_pred)
        
        if best_score is None or curr_score > best_score:
            best_score = curr_score
            best_params = {
                'learning_rate': lr,
                'depth': md,
                'min_data_in_leaf': msl
            }
    return best_params, best_score

#### Возраст

In [17]:
best_params, best_score = get_best_params(y_age_train, y_age_eval)
best_params, best_score

  0%|          | 0/45 [00:00<?, ?it/s]

({'learning_rate': np.float64(0.25075), 'depth': 7, 'min_data_in_leaf': 1},
 np.float64(0.7351086194814296))

In [18]:
cb_model = CatBoostClassifier(
    iterations=100,
    depth=7,
    learning_rate=np.float64(0.25075),
    min_data_in_leaf=1,
    loss_function='MultiClass',
    verbose=False
)
cb_model.fit(X_train, y_age_train)
y_pred = cb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_age_test, y_pred))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_model.feature_importances_, features, 10)):
    print(str(i + 1) + ".", name)

Accuracy: 0.7238335435056746
Most important features:
1. ovsyanochan
2. leprum
3. mudakoff
4. dayvinchik
5. 4ch
6. styd.pozor
7. rhymes
8. bestad
9. fuck_humor
10. rapnewrap


#### Пол

In [19]:
best_params, best_score = get_best_params(y_sex_train, y_sex_eval)
best_params, best_score

  0%|          | 0/45 [00:00<?, ?it/s]

({'learning_rate': np.float64(0.75025), 'depth': 4, 'min_data_in_leaf': 1},
 np.float64(0.8682550805886475))

In [20]:
cb_model = CatBoostClassifier(
    iterations=100,
    depth=4,
    learning_rate=np.float64(0.75025),
    min_data_in_leaf=1,
    loss_function='MultiClass',
    verbose=False
)
cb_model.fit(X_train, y_sex_train)
y_pred = cb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_sex_test, y_pred))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_model.feature_importances_, features, 10)):
    print(str(i + 1) + ".", name)

Accuracy: 0.8511979823455234
Most important features:
1. 40kg
2. mudakoff
3. modnailru
4. girlmeme
5. zerofat
6. thesmolny
7. i_d_t
8. igm
9. femalemem
10. be.beauty
